In [1]:
!pip install transformers
!pip install tensorboardx
!pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 4.5 MB/s eta 0:00:0000:01


In [2]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backend_bases import RendererBase
from scipy import signal
from scipy.io import wavfile
import os
from PIL import Image
from scipy.fftpack import fft
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
import csv
from fastai import *
from fastai.vision.all import *
import PIL
from io import BytesIO
# from file_util import *

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#-*- coding: utf-8 -*-


import os
import chardet

'''
dirname        : path that need to be searched
ret                : files in the dirname (recursive)
list_avoid_dir : dirname need to be skipped
usage           : 
    list_files = []
    file_search(dirname, list_files):   
'''
def file_search(dirname, ret, list_avoid_dir=[]):
    
    filenames = os.listdir(dirname)
    
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)

        if os.path.isdir(full_filename) :
            if full_filename.split('/')[-1] in list_avoid_dir:
                continue
            else:
                file_search(full_filename, ret, list_avoid_dir)
            
        else:
            ret.append( full_filename )          

            

'''
filename : filename (inc. path) that will be inspected
'''
def find_encoding(filename):
    rawdata = open(filename, 'rb').read()
    result = chardet.detect(rawdata)
    charenc = result['encoding']    
    return charenc
            
'''
dir_name : dir_name (inc. path) that will be created ( full-path name )
'''
def create_folder(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

In [4]:

list_files = []
for x in range(5):
    sess_name = 'Session' + str(x+1)
    path = '/kaggle/input/iemocapfullrelease/IEMOCAP_full_release/'+ sess_name + '/sentences/wav/'
    file_search(path, list_files)
    list_files = sorted(list_files)
    print (sess_name + ", #sum files: " + str(len(list_files)))
#extract_feature( list_files, out_file )

Session1, #sum files: 1820
Session2, #sum files: 3633
Session3, #sum files: 5769
Session4, #sum files: 7873
Session5, #sum files: 10043


In [5]:
import pandas as pd
df=pd.read_csv('/kaggle/input/emotion-recognition/processed_data.csv')

In [6]:
df['label'] = pd.to_numeric(df['label'])

In [9]:
df.tail()

,Unnamed: 0,sessionID,text,label
10034,10034,Ses05M_script03_2_M041,"You are a vile tempered, wicked living, evil little beast. And I hope I never set eyes on you ever again.",0
10035,10035,Ses05M_script03_2_M042,"Oh, you're not going like this.",0
10036,10036,Ses05M_script03_2_M043,"[GARBAGE] No, you're not.",0
10037,10037,Ses05M_script03_2_M044,oh! Marry you again? I wouldn't marry you again if you came crawling to me on bended knee.,0
10038,10038,Ses05M_script03_2_M045,You're a wicked little vampire. And I pray to God I never set eyes on you again as long as I live.,0


# *EXTACT THE MFCC FEATURE USING LIBROSA*

In [7]:
def audio2spectrogram(file_path):
    y, sr = librosa.load(file_path, sr=None)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)
    librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')
    fig1 = plt.gcf()
    plt.axis('off')

    buffer = BytesIO()
    fig1.savefig(buffer, format='jpg')
    buffer.seek(0)

    # Read the bytes and display the image
    image = PIL.Image.open(buffer)
    
    preprocess = transforms.Compose([
        transforms.Resize(360),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.7137, 0.6628, 0.6519], std=[0.2970, 0.3017, 0.2979]),
    ])
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
    
    buffer.close()
    return input_batch

In [ ]:
no_rows=len(list_files)
index=0
sprectrogram_shape=[]
docs = []
bookmark=0
extraLabel=0
for everyFile in list_files:
  if(everyFile.split('/')[-1].endswith('.wav')):
    if(index <= 5600):
        index += 1
        continue
    filename=everyFile.split('/')[-1].strip('.wav')
    lable=df.loc[df['sessionID']==filename]['label'].values[0]
    text=df.loc[df['sessionID']==filename]['text'].values[0]
    print('label',lable)
    if(lable!=-1):
      #sprectrogram_shape.append(audio2spectrogram(everyFile))
      input_batch=audio2spectrogram(everyFile)
      docs.append({
         'fileName':everyFile.split('/')[-1].strip('.wav'),
         'text':text,
         'sprectrome':input_batch,
         'label':lable
              })
      index+=1
      print('index',index)
    else:
      extraLabel=extraLabel+1
      print('extraLabel',extraLabel)

In [9]:
import pickle


# Save list to a file using pickle
# with open('docs_list_five.pkl', 'wb') as file:
#     pickle.dump(docs, file)

# # Load list from the file
loaded_docs_zero = []
loaded_docs_one = []
loaded_docs_two = []
loaded_docs_three = []
loaded_docs_four = []
loaded_docs_five = []
with open('/kaggle/input/emotion-recognition/docs_list.pkl', 'rb') as file:
    loaded_docs_zero = pickle.load(file)
with open('/kaggle/input/emotion-recognition/docs_list_one.pkl', 'rb') as file:
    loaded_docs_one = pickle.load(file)
with open('/kaggle/input/emotion-recognition/docs_list_two.pkl', 'rb') as file:
    loaded_docs_two = pickle.load(file)
with open('/kaggle/input/emotion-recognition/docs_list_three.pkl', 'rb') as file:
    loaded_docs_three = pickle.load(file)
with open('/kaggle/input/emotion-recognition/docs_list_four.pkl', 'rb') as file:
    loaded_docs_four = pickle.load(file)
with open('/kaggle/input/emotion-recognition/docs_list_five.pkl', 'rb') as file:
    loaded_docs_five = pickle.load(file)

In [10]:
docs = loaded_docs_zero + loaded_docs_one + loaded_docs_two + loaded_docs_three + loaded_docs_four + loaded_docs_five

In [11]:
len(docs)

6226

In [12]:
outputs_text= []
def hook_text(module, input, output):
    outputs_text.clear()
    outputs_text.append(output)
    return None
     

outputs_audio= []
def hook_audio(module, input, output):
    outputs_audio.clear()
    outputs_audio.append(output)
    return None

# New Model

In [15]:
from transformers import BertModel, BertTokenizer

In [16]:
class CombinedAudioTextModel(nn.Module):
    def __init__(self, num_classes=4):
        super(CombinedAudioTextModel, self).__init__()
        self.num_classes=num_classes
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.text_model=torch.load('/kaggle/input/emotion-recognition/model_text.pt')
        self.audio_model=torch.load('/kaggle/input/emotion-recognition/audio_based_er_resnet34.pkl').model

        self.text_model.bert.pooler.register_forward_hook(hook_text)
        self.audio_model[0].register_forward_hook(hook_audio)


        for param in self.text_model.parameters():
          param.requires_grad = False
        for param in self.audio_model.parameters():
          param.requires_grad = False

        self.dropout = nn.Dropout(.5)
        self.linear = nn.Linear(1280, num_classes)

        self.softmax = nn.Softmax(dim=1)

    def forward(self,text,audio):
        self.text_model(text)
        try:
            self.audio_model(audio)
        except:
            print("Exception due to batch in training")
        audio_embed=outputs_audio[0]
#         print(audio_embed)
        text_embed=outputs_text[0]
#         print(audio_embed, text_embed)
        audio_embed=torch.flatten(audio_embed, start_dim=2)
        audio_embed=torch.sum(audio_embed, dim=2)
        concat_embded=torch.cat((text_embed,audio_embed),1)
        x=self.dropout(concat_embded)
        x=self.linear(x)
        return x

In [17]:
model=CombinedAudioTextModel(num_classes=4)
model.to('cuda')

CombinedAudioTextModel(
  (text_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_

# check trainable model parameter

In [18]:
for name, param in model.named_parameters():
      if(param.requires_grad):
        print(name)
      else:
        print('no grad',name)

no grad text_model.bert.embeddings.word_embeddings.weight
no grad text_model.bert.embeddings.position_embeddings.weight
no grad text_model.bert.embeddings.token_type_embeddings.weight
no grad text_model.bert.embeddings.LayerNorm.weight
no grad text_model.bert.embeddings.LayerNorm.bias
no grad text_model.bert.encoder.layer.0.attention.self.query.weight
no grad text_model.bert.encoder.layer.0.attention.self.query.bias
no grad text_model.bert.encoder.layer.0.attention.self.key.weight
no grad text_model.bert.encoder.layer.0.attention.self.key.bias
no grad text_model.bert.encoder.layer.0.attention.self.value.weight
no grad text_model.bert.encoder.layer.0.attention.self.value.bias
no grad text_model.bert.encoder.layer.0.attention.output.dense.weight
no grad text_model.bert.encoder.layer.0.attention.output.dense.bias
no grad text_model.bert.encoder.layer.0.attention.output.LayerNorm.weight
no grad text_model.bert.encoder.layer.0.attention.output.LayerNorm.bias
no grad text_model.bert.encoder.

In [19]:
import random
random.shuffle(docs)
random.shuffle(docs)
random.shuffle(docs)
total_length=len(docs)
train_length=int(.9*total_length)
train_list=docs[0:train_length]
test_list=docs[train_length:]
print('no of items for train ',len(train_list))
print('no of items for test ',len(test_list))

no of items for train  5603
no of items for test  623


In [20]:
# train_list = train_list[:len(train_list)-1]
len(train_list)

5603

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
label1=train_list[12]['label']
text=train_list[12]['text']
input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
label1=torch.tensor([label1])
sprectrome=train_list[12]['sprectrome']
model.to('cpu')
model.eval()
with torch.no_grad():
    output = model(input_ids,sprectrome)
    #output.squeeze().shape
    #output=torch.flatten(output, start_dim=2)
    #print(output.shape)
    #output=torch.sum(output, dim=2)
    print(output)
     

tensor([[-10.2130,  61.0141,   2.6654,  14.6554]])


# Testing data loader

In [ ]:
df = pd.DataFrame(train_list)

In [ ]:
df.head(1)

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]

        # Assuming 'spectrogram' is the column with tensors
        sprectrome = row['sprectrome']

        # Assuming 'text' is the column with text data
        text = row['text']

        # Assuming 'label' is the column with labels
        label = row['label']

        # Convert your data to PyTorch tensors
        # You might need to adjust this depending on your actual data structure
        sprectrome = torch.tensor(sprectrome, dtype=torch.float32)
        # Convert text to tensors using your tokenizer or other appropriate method

        return {'sprectrome': sprectrome, 'text': text, 'label': label}

# Example usage:
my_dataset = MyDataset(df)

In [ ]:
my_dataset[0]

In [ ]:
from torch.utils.data import DataLoader

# Use DataLoader with shuffle=True
train_dataloader = DataLoader(my_dataset, batch_size=4, shuffle=True)

# Training loop
for epoch in range(1):
    for batch in train_dataloader:
        print(batch)
        break

# optimizer

In [22]:
import torch.optim as optim
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
writer = SummaryWriter(log_dir='/kaggle/working/content/')

In [23]:
docs[12]["text"]

"That's the wind."

In [24]:
total_steps = 1
NUM_EPOCHS=2
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.train()
model.to('cuda')
for epoch in range(NUM_EPOCHS):
  lr_scheduler.step()
  random.shuffle(train_list)
  for every_trainlist in train_list:
    label1=every_trainlist['label']
    text=every_trainlist['text']
    label1=torch.tensor([label1])
    sprectrome=every_trainlist['sprectrome']
#     testing_audio_input = sprectrome
#     if(sprectrome.shape[2]>65):
    optimizer.zero_grad()
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0) 
    sprectrome = sprectrome.to('cuda')
#     print(sprectrome)
#     print(sprectrome.size())
    label1=label1.to('cuda')
    input_ids=input_ids.to('cuda')
    output = model(input_ids,sprectrome)
    #print('softmax output ',output)
    loss = criterion(output, label1)
    #print('label1',label1)
    print('loss',loss.item())
    loss.backward()
    optimizer.step()
    _, preds = torch.max(output, 1)
    accuracy = torch.sum(preds == label1)
    print('accuracy.item()',accuracy.item())
    #print('preds',preds)
    if total_steps % 10 == 0:
        with torch.no_grad():
          _, preds = torch.max(output, 1)
          accuracy = torch.sum(preds == label1)
          #print('Epoch: {} \tStep: {} \tLoss: {:.4f} \tAcc: {}'.format(epoch + 1, total_steps, loss.item(), accuracy.item()))
          writer.add_scalar('loss', loss.item(), total_steps)
          writer.add_scalar('accuracy', accuracy.item(), total_steps)                     
        total_steps+=1

Exception due to batch in training
loss 0.0
accuracy.item() 1
Exception due to batch in training
loss 0.0
accuracy.item() 1
Exception due to batch in training
loss 149.59124755859375
accuracy.item() 0
Exception due to batch in training
loss 209.9805145263672
accuracy.item() 0
Exception due to batch in training
loss 109.01388549804688
accuracy.item() 0
Exception due to batch in training
loss 0.022282473742961884
accuracy.item() 1
Exception due to batch in training
loss 164.79359436035156
accuracy.item() 0
Exception due to batch in training
loss 0.0
accuracy.item() 1
Exception due to batch in training
loss 91.8606185913086
accuracy.item() 0
Exception due to batch in training
loss 0.0
accuracy.item() 1
Exception due to batch in training
loss 226.51400756835938
accuracy.item() 0
Exception due to batch in training
loss 0.0
accuracy.item() 1
Exception due to batch in training
loss 71.25639343261719
accuracy.item() 0
Exception due to batch in training
loss 5.104355335235596
accuracy.item() 0


# Trying resnet34 alone

In [ ]:
df.head(40)

In [ ]:
audio_model = torch.load('/kaggle/input/emotion-recognition/audio_based_er_resnet34.pkl').model

In [ ]:
audio_model

In [ ]:
import matplotlib as mpl
# plt.rcParams['figure.dpi'] = 100

In [ ]:
from io import BytesIO

base_path = "/kaggle/input/iemocapfullrelease/IEMOCAP_full_release/Session1/sentences/wav/Ses01F_impro02"
y, sr = librosa.load(base_path + "/Ses01F_impro02_F003.wav", sr=None)
S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
log_S = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')
fig1 = plt.gcf()
plt.axis('off')

buffer = BytesIO()
fig1.savefig(buffer, format='jpg')
buffer.seek(0)

# Read the bytes and display the image
image = PIL.Image.open(buffer)

In [ ]:
img = PIL.Image.open("/kaggle/working/Ses01F_impro02_F003.jpg")
# audio_model

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(360),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.7137, 0.6628, 0.6519], std=[0.2970, 0.3017, 0.2979]),
])
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

In [ ]:
output = audio_model(input_batch)

In [ ]:
output

# Testing audio model

In [ ]:
docs[0]

In [ ]:
audio_model = torch.load('/kaggle/input/emotion-recognition/audio_based_er_resnet34.pkl').model
audio_model.to('cpu')

In [ ]:
audio_model[0].register_forward_hook(hook_audio)
for param in audio_model.parameters():
    param.requires_grad = False

In [ ]:
audio_model(docs[0]['sprectrome'])

In [ ]:
outputs_audio[0].size()

In [ ]:
testing_audio_input.size()

In [ ]:
audio_model(testing_audio_input)

In [ ]:
outputs_audio[0].size()

# Testing text model

In [ ]:
text_model=torch.load('/kaggle/input/emotion-recognition/model_text.pt')

In [ ]:
text_model

In [ ]:
sample_text = train_list[12]["text"]
input_ids = torch.tensor(tokenizer.encode(sample_text, add_special_tokens=True)).unsqueeze(0) 
text_model.bert.pooler.register_forward_hook(hook_text)
for param in text_model.parameters():
  param.requires_grad = False
text_model.to('cpu')
text_model.eval()

In [ ]:
text_model(input_ids)

In [ ]:
outputs_text[0].size()

# Trying concatenation

In [ ]:
outputs_audio[0]

In [ ]:
outputs_text[0]

In [ ]:
audio_embed = torch.flatten(outputs_audio[0], start_dim=2)
audio_embed = torch.sum(audio_embed, dim=2)

In [ ]:
audio_embed.size()

In [ ]:
concat_embed = torch.cat((outputs_text[0], audio_embed), 1)

In [ ]:
outputs_text[0].size()

In [ ]:
concat_embed.size()

# Testing the model

In [25]:
y_actu=[]
y_pred=[]
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.to('cpu')
model.eval()
for every_test_list in test_list:
    label1=every_test_list['label']
    label1=torch.tensor([label1])
    sprectrome=every_test_list['sprectrome']
    text=every_test_list['text']
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
    with torch.no_grad():
      if(sprectrome.shape[2]>65):
        #sprectrome = sprectrome.to('cuda')
        #label1=label1.to('cuda')
        output = model(input_ids,sprectrome)
        _, preds = torch.max(output, 1)
        y_actu.append(label1.numpy()[0])
        y_pred.append(preds.numpy()[0])

In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_actu, y_pred)

array([[146,   0,   1,   4],
       [ 22, 118,  23,   9],
       [  2,   0, 109,   5],
       [ 10,   4,  15, 155]])

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_actu,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.97      0.88       151
           1       0.97      0.69      0.80       172
           2       0.74      0.94      0.83       116
           3       0.90      0.84      0.87       184

    accuracy                           0.85       623
   macro avg       0.85      0.86      0.84       623
weighted avg       0.87      0.85      0.85       623



In [28]:
torch.save(model, '/kaggle/working/Combined_model_resnet_bert_new.pt')